In [1]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from funpymodeling.exploratory import freq_tbl
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('Roma.csv')

In [3]:
df= df.fillna(method="bfill")
df= df.fillna(method="ffill")
df = df.bfill()
df= df.ffill()

C:\Users\HP\AppData\Local\Temp\ipykernel_5072\1730909967.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method="bfill")
C:\Users\HP\AppData\Local\Temp\ipykernel_5072\1730909967.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df= df.fillna(method="ffill")


**Transformando variables a tipo dicotómicas**

In [4]:

#bathrooms_text
df["bathroomsd"] = df["bathrooms_text"].astype(str).apply(lambda x: 1 if "2" in x or "3" in x or "4" in x else 0)
#room_type Entire home
df["E_room_typed"] = df["room_type"].apply(lambda x: 1 if x == "Entire home/apt" else 0)
#room_type Private home
df["P_room_typed"] = df["room_type"].apply(lambda x: 1 if x == "Private room" else 0)
#room_type Hotel home
df["H_room_typed"] = df["room_type"].apply(lambda x: 1 if x == "Hotel room" else 0)
#instant_bookabled
df["instant_bookabled"] = df["instant_bookable"].apply(lambda x: 1 if x == "t" else 0)
#Price
df["price"] = df["price"].replace('[\$,]', '', regex=True).astype(float)
Mprice = df["price"].median()
df["D_price"] = df["price"].apply(lambda x: 1 if x >= Mprice else 0)
#host_total_listings_count
Mlistings = df["host_total_listings_count"].median()
df["D_host_listings"] = df["host_total_listings_count"].apply(lambda x: 1 if x >= Mlistings else 0)
#host_total_listings_count
Maccuracy = df["host_total_listings_count"].median()
df["D_accuracy"] = df["review_scores_accuracy"].apply(lambda x: 1 if x >= Maccuracy else 0)
#availability_30
Mavailability = df["availability_30"].median()
df["D_availability"] = df["availability_30"].apply(lambda x: 1 if x >= Mavailability else 0)
#availability_90
Mavailability90 = df["availability_90"].median()
df["D_availability90"] = df["availability_90"].apply(lambda x: 1 if x >= Mavailability90 else 0)
#number_of_reviews
Mreviews = df["number_of_reviews"].median()
df["D_reviews"] = df["number_of_reviews"].apply(lambda x: 1 if x >= Mreviews else 0)
#review_scores_rating
Mrating = df["review_scores_rating"].median()
df["D_rating"] = df["review_scores_rating"].apply(lambda x: 1 if x >= Mrating else 0)
#review_scores_rating
Maccommodates = df["accommodates"].median()
df["D_accommodates"] = df["accommodates"].apply(lambda x: 1 if x >= Maccommodates else 0)



<>:12: SyntaxWarning: invalid escape sequence '\$'
<>:12: SyntaxWarning: invalid escape sequence '\$'
C:\Users\HP\AppData\Local\Temp\ipykernel_5072\2275433636.py:12: SyntaxWarning: invalid escape sequence '\$'
  df["price"] = df["price"].replace('[\$,]', '', regex=True).astype(float)


In [5]:
df.head(5)

,Unnamed: 0,id,name,description,neighborhood_overview,host_id,host_name,host_since,host_location,host_response_time,...,H_room_typed,instant_bookabled,D_price,D_host_listings,D_accuracy,D_availability,D_availability90,D_reviews,D_rating,D_accommodates
0,0,2737.0,"Elif's room in cozy, clean flat.",10 min by bus you can get to Piazza Venezia or...,It used to be an industrial area until late 80...,3047,Elif,18/09/2008,"Rome, Italy",within an hour,...,0,1,0,1,0,1,1,0,1,0
1,1,3079.0,Cozy apartment (2-4)with Colisseum view,With the view of the Colisseum from the front ...,Monti neighborhood is one of the best areas in...,3504,Laura,08/10/2008,"Rome, Italy",within a few hours,...,0,1,1,1,0,0,0,0,0,1
2,3,12398.0,Casa Donatello - Home far from Home,Casa Donatello is a newly renovated two-bedroo...,You are at 15 minutes walking distance from hi...,11756,Gea,30/03/2009,"Rome, Italy",within an hour,...,0,0,1,1,0,1,1,1,1,1
3,4,19965.0,S.Peter's Basilica 5 MIN WALK two large bedrooms,Bright and quiet two bedrooms apartment 3rd fl...,Prati is a famous neighbourhood (rione of Rome...,75450,Sasa,30/01/2010,"Rome, Italy",within an hour,...,0,0,1,1,0,1,1,1,0,1
4,5,50333.0,Bright&Cosy Penthouse Overlooking St. Peter's ...,"Very bright and comfortable, impressive views ...","There are plenty of shops nearby, small restau...",230736,Sita Alessandra,10/09/2010,"Rome, Italy",within an hour,...,0,0,1,0,0,1,1,1,1,0


**CASO 1: accommodates, bathrooms, room_type**

In [6]:
#Declaramos las variables dependientes e independiente para las regresion Logisitica 
Vars_Indep = df[['D_accommodates', 'bathroomsd', 'E_room_typed']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

#Se escalan los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [7]:
#Verifico la matriz de Confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[701 379]
 [406 733]]


In [8]:
#Caulculo de la precisuin del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.6591726618705036


In [9]:
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.6462370437133844


In [10]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.6435469710272168


**CASO 2:** 
**price, host_listings, accuracy**

In [11]:
#Declaramos las variables dependientes e independiente para las regresion Logisitica 
Vars_Indep = df[['D_price', 'D_host_listings', 'D_accuracy']]
Var_Dep = df['D_availability']
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

#Se escalan los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [12]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[610 362]
 [510 737]]


In [13]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.6706096451319381


In [14]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.6070301937809824


In [15]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.591018444266239


**CASO 3:** 
**availability, number_of_reviews, review_scores_rating**

In [16]:
#Declaramos las variables dependientes e independiente para las regresion Logisitica 
Vars_Indep = df[['D_availability', 'D_reviews', 'D_rating']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

#Dividimos el conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state= None)

#Se escalan los datos
escalar = StandardScaler()
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizaremos 
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

#Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()

algoritmo.fit(X_train, y_train)

y_pred = algoritmo.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [17]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[738 369]
 [538 574]]


In [18]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.6086956521739131


In [19]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5912573231185219


In [20]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.5161870503597122


**CASO 4:** 
**bathrooms, number_of_reviews, price**

In [21]:
# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['bathroomsd', 'D_reviews', 'D_price']]
Var_Dep = df['D_availability']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 0, 0, ..., 0, 0, 1], dtype=int64)

In [22]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[613 351]
 [503 752]]


In [23]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.6817769718948323


In [24]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.6151419558359621


In [25]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.599203187250996


**CASO 5:** 
**accommodates, review_scores_rating, availability_30**

In [26]:
# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['D_accommodates', 'D_reviews', 'D_availability']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
escalar = StandardScaler()
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 0, 0, ..., 1, 0, 0], dtype=int64)

In [27]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[693 418]
 [344 764]]


In [28]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.6463620981387479


In [29]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.6566020730058585


In [30]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.6895306859205776


**CASO 6:** 
**instant_bookable, review_scores_rating, availability_90**

In [31]:

# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['instant_bookabled', 'D_reviews', 'D_availability90']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 1, 1, ..., 1, 0, 1], dtype=int64)

In [32]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[641 454]
 [458 666]]


In [33]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5946428571428571


In [34]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5890040558810274


In [35]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.5925266903914591


**CASO 7:** 
**bathrooms, availability_30, host_listings**

In [36]:

# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['bathroomsd', 'D_availability', 'D_host_listings']]
Var_Dep = df['D_reviews']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 0, 0, ..., 1, 0, 1], dtype=int64)

In [37]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[576 546]
 [421 676]]


In [38]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5531914893617021


In [39]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5642181162685894


In [40]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.6162260711030082


**CASO 8:** 
**price, accommodates, review_scores_rating**

In [41]:

# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['D_price', 'D_accommodates', 'D_rating']]
Var_Dep = df['D_reviews']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([0, 1, 0, ..., 1, 1, 0], dtype=int64)

In [42]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
exactitud = accuracy_score(y_test, y_pred)

print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[625 484]
 [474 636]]


In [43]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5678571428571428


In [44]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5682739972960793


In [45]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.572972972972973


**CASO 9:** 
**bathrooms, review_scores_accuracy, review_scores_rating**

In [46]:

# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['bathroomsd', 'D_accuracy', 'D_rating']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([1, 1, 0, ..., 0, 0, 0], dtype=int64)

In [47]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
exactitud = accuracy_score(y_test, y_pred)

print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[984 109]
 [768 358]]


In [48]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.7665952890792291


In [49]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.604776926543488


In [50]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.31793960923623443


**CASO 10:** 
**availabilit_90, host_total_listings_count, number_of_reviews**

In [51]:
# Declaramos las variables dependientes e independientes para la regresión logística
Vars_Indep = df[['D_availability90', 'D_host_listings', 'D_reviews']]
Var_Dep = df['D_price']
X = Vars_Indep
y = Var_Dep

# Dividimos el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

# Se escalan los datos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

# Definimos el algoritmo a utilizar
algoritmo = LogisticRegression()
algoritmo.fit(X_train, y_train)

# Hacemos las predicciones
y_pred = algoritmo.predict(X_test)
y_pred


array([0, 0, 1, ..., 1, 0, 0], dtype=int64)

In [52]:
#Verifico la matriz de Confusion
matriz = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
exactitud = accuracy_score(y_test, y_pred)

print('Matriz de Confusion:')
print(matriz)

Matriz de Confusion:
[[625 464]
 [479 651]]


In [53]:
#Caulculo de la precisuin del modelo
precision = precision_score(y_test, y_pred, average='binary')
print("Precision del modelo: ")
print(precision)

Precision del modelo: 
0.5838565022421525


In [54]:
exactitud = accuracy_score(y_test, y_pred)
print("Exactitud del modelo: ")
print(exactitud)


Exactitud del modelo: 
0.5750337990085624


In [55]:
#Calculo la sensibilidad del modelo
sensibilidad = recall_score(y_test, y_pred, average="binary")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.5761061946902655
